In [1]:
using Gurobi, JuMP   #JuMP是一个优化库(可以在里面插入优化的库，比如Gurobi) Import everything 
using DataFrames, GLM
using Plots
using Distributions
using Random
using LinearAlgebra
using DelimitedFiles

In [2]:
# Multivariable Newsvendor Problem
function GenData(n,m,a,a2,b,b2,c,c2,r,r2,s,form="newsvendor",seed=1)
	Random.seed!(seed)   #Random: Support for generating random numbers.   Random.rand!-Function是的，(seed)可以将随机数固定
	#Drawing random decision data
	x = rand(n)  #x是[0,1]之间的随机数，所以我们产生的data points是随机的
    z = rand(n)
	#Drawing random outcome data
	#y,y_min,y_max = Outcome(x,a,b,c,r,s,form )
    y,y_min,y_max = Outcome(x,z,a,a2,b,b2,c,c2,r,r2,s,form)     
	#Setting all possible values for decisions
    d_1 = collect(0.1:1/m:0.9)     ### How to decide the step size here ????  Make each interval have data points and avoid boundary issues
    d_2 = collect(0.1:1/m:0.9)
    d = []
    d2 = []
    for i = 1:length(d_1)
        num = d_1[i]   # The number we want to push in this iteration
        for j = 1:length(d_1)
            push!(d,num)   # We need to push this number for length(d) times in this iteration
        end
    end
            
    for j = 1:length(d_1)
        append!(d2,d_2)   # We use append() to add a list at the end of the former list
    end
            
        
	return x,z,y,y_min,y_max,d,d2
end

#Function for simulating outcome. 拟合各种函数结果
function Outcome(x,z,a,a2,b,b2,c,c2,r,r2,s,form="newsvendor")
	n = length(x)
    y = zeros(n)
    #Simulating the random outcome for each random decision
    for i = 1:n
        if form == "flat"
            y[i] = a + s*randn()[1]
        elseif form == "flat-double"
            y[i] = a + s*randn()[1]
            if 0.5 <= x[i] <= 0.75
                y[i] = r*a + s*randn()[1]   #这里有随机项，所以对于相同的x值，y值也有很小的差别
            end
        elseif form == "linear"
            y[i] = a + b*x[i] + s*randn()[1]
        elseif form == "linear-double"
            y[i] = a + b*x[i] + s*randn()[1]
            if 0.5 <= x[i]
                y[i] = r*a + a*(1-r)*x[i] + s*randn()[1]
            end
        elseif form == "quadratic"
            y[i] = a + b*x[i] + c*x[i]^2 + s*randn()[1]
        elseif form == "quadratic-double"
            y[i] = a + b*x[i] + c*x[i]^2 + s*randn()[1]   #此处需要数字而不是数组，故需要将randn() 转化为randn()[1]
            if 0.5 <= x[i]
                y[i] = r*a + (3*a*(1-r)+2*b+c)*x[i] + (-2*a*(1-r)-2*b-c)*x[i]^2 + s*randn()[1]    #r具体是多少？如何使得函数在边界点处相等？可以用公式计算出r的表达式
            end
        elseif form == "newsvendor"
        	demand = a + s*randn()[1]
            demand2 = a2 + s*rand()[1]
            # We have new variables in newsvendor problem
        	y[i] = -(c*x[i] + r*b*max(demand-x[i],0) + b*max(x[i]-demand,0) + c2*z[i] + r2*b2*max(demand2-z[i],0) + b2*max(z[i]-demand2))
        elseif form == "supplychain-normal"
        	y[i] = x[i] * (quantile(Normal(c,r),(a-x[i])/(a+b))) + s*randn()[1]
        elseif form == "supplychain-pareto"
        	y[i] = x[i] * (quantile(Pareto(c,r),(a-x[i])/(a+b))) + s*randn()[1]
        end		       
    end
    #Scaling the outcome to unit scale
    y_min = minimum(y)
    y_max = maximum(y)
    y = (y - ones(n)*y_min)./(ones(n)*(y_max-y_min))   #将y归一化
	return y,y_min,y_max
end


Outcome (generic function with 2 methods)

In [3]:
# Multivariable Newsvendor Problem (Function for computing result  )
function Result(y_min,y_max,x,z,a,a2,b,b2,c,c2,r,r2,form="newsvendor")
	#Computing the expected outcome for given decision
    if form == "flat"
        y = a
    elseif form == "flat-double"
        y = a
        if 0.5 <= x <= 0.75
            y = r*a               ### What is the value of r??????
        end
    elseif form == "linear"
        y = a + b*x
    elseif form == "linear-double"
        y = a + b*x
        if 0.5 <= x
            y = r*a + a*(1-r)*x
        end
    elseif form == "quadratic"
        y = a + b*x + c*x^2
    elseif form == "quadratic-double"
        y = a + b*x + c*x^2
        if 0.5 <= x
            y = r*a + (3*a*(1-r)+2*b+c)*x + (-2*a*(1-r)-2*b-c)*x^2
        end
    elseif form == "newsvendor"
    	demand = a
        demand2 = a2
    	y = -(c*x + r*b*max(demand-x,0) + b*max(x-demand,0) + c2*z + r2*b2*max(demand2-z,0) + b2*max(z-demand2))
    elseif form == "supplychain-normal"
        y = x * (quantile(Normal(c,r),(a-x)/(a+b)))
    elseif form == "supplychain-pareto"
        y = x * (quantile(Pareto(c,r),(a-x)/(a+b)))
    end
    #Scaling the outcome to unit scale
    y = (y-y_min)/(y_max-y_min)
	return y
end

Result (generic function with 2 methods)

In [4]:
#Function for jointly estimating and optimizing
function EWO(x,Z_data,y,d,d2,kappa,lambda,M,showoutput=false)   # Where do we define x and d here????
    n = Int(length(x))   # Number of the data points
    #n = length(Z)
    m = Int(sqrt(length(d)))  # One dimension of binary variable
    #Defining optimization model
	model = Model()
	set_optimizer(model, Gurobi.Optimizer)      #调用Gurobi里面的optimizer? Yep
	set_optimizer_attributes(model, "OutputFlag" => 1)   #??????  This arrow means equal to zero
    #Defining variables
    #println("Its correct 1!")
    @variable(model, z[1:m,1:m], Bin)   
    @variable(model, a)     ### 定义变量语法（general way）
    @variable(model, b)
    #@variable(model, c)
    @variable(model, b2)
    @variable(model, u[1:n], Bin)   #u[1:n]都是binary variable
    @variable(model, v[1:n], Bin)
    @variable(model, w[1:n], Bin)
    @variable(model, f[1:n], Bin)   #f[1:n]都是binary variable
    @variable(model, g[1:n], Bin)
    @variable(model, h[1:n], Bin)
    #@variable(model, z[1:m,1:m], Int)   
    #println("Its correct 2!")
    @variable(model, s[1:m,1:m] >= 0)
    @variable(model, t[1:n] >= 0)
    @variable(model, Z[1:m], Bin)
    @variable(model, Z2[1:m], Bin)
	#Defining objective
    @objective(model, Max, sum(s[i,j] for i=1:m for j=1:m) - lambda * sum(t[i] for i=1:n))   # Updated
    #Defining constraints
    # Task: Redo the constraint section!!!
    @constraints(model, begin
        decision, sum(z[i,j] for i=1:m for j=1:m) == 1   #decision here is the name of the constraint Updated
        revenue_passive[i=1:m,j=1:m], s[i,j] <= a+b*d[i]+b2*d2[j]  #Updated
        revenue_active[i=1:m,j=1:m], s[i,j] <= M*z[i,j]   #Updated
        data[i=1:n], u[i] >= v[i] + w[i] - 1   #Updated
        Z_value[k=1:m], Z[k]==sum(z[k,i] for i=1:m)   #Updated
        Z2_value[j=1:m], Z2[j]==sum(z[d,j] for d=1:m)   #Updated
        data_upper[i=1:n], x[i] - sum(d[k]*Z[k] for k = 1:m ) >= kappa - M*v[i]   ### Updated Meaning of this constraints? It's a linear programming problem, but the equivalents are hard to write
        data_lower[i=1:n], sum(d[k]*Z[k] for k = 1:m ) - x[i] >= kappa - M*w[i]   #Updated
        #Z_value[k=1:m], Z[k]==sum(z[k,i] for i=1:m)   #Updated
        #Z2_value[j=1:m], Z2[j]==sum(z[d,j] for d=1:m)   #Updated
        data2[i=1:n], f[i] >= g[i] + h[i] - 1   #Updated
        data2_upper[i=1:n], Z_data[i] - sum(d2[j]*Z2[j] for j=1:m) >= kappa - M*g[i]   ### Updated Meaning of this constraints? It's a linear programming problem, but the equivalents are hard to write
        data2_lower[i=1:n], sum(d2[j]*Z2[j] for j=1:m) - Z_data[i] >= kappa - M*h[i]   #Updated
        estimation_upper[i=1:n], t[i] >= y[i]-a-b*x[i]-b2*Z_data[i]-M*(1-u[i]+1-f[i])   #Updated
        estimation_lower[i=1:n], t[i] >= -y[i]+a+b*x[i]+b2*Z_data[i]-M*(1-u[i]+1-f[i])   #Updated
    end)

    #Optimizing model
    optimize!(model)
    obj_val = objective_value(model)
    time_val = solve_time(model)
    a_val = value.(a)
    b_val = value.(b)
    #c_val = value.(c)
    b2_val = value(b2)
    z_val = value.(z)
    return obj_val,time_val,a_val,b_val,b2_val,z_val
end

EWO (generic function with 2 methods)

In [5]:
@variable(model, X[1:2, 1:2],Bin)

LoadError: UndefVarError: model not defined

In [6]:
#Selecting setup
#setup = ["flat-double",100,0,0,4,1,0.05,1]
#setup = ["linear-double",100,100,0,0.75,1,0.05,1]
#setup = ["quadratic-double",100,100,-100,0.5,1,0.05,0.1]
setup = ["newsvendor",0.3,0.8,1,2,1,3,0.75,0.5,0.1,0.05,0.1]  # rho: the degree of scaling
#setup = ["supplychain-normal",1,1,4,3,0.1,0.05,0.1]
#setup = ["supplychain-pareto",1,1.5,0.25,1,0.1,0.05,0.1]

#Setting parameters    
form_data = setup[1]
alpha = setup[2]
alpha2 = setup[3]
beta = setup[4]
beta2 = setup[5]
gamma = setup[6]
gamma2 = setup[7]
rho = setup[8]
rho2 = setup[9]
sigma = setup[10]   #相当于前面的s
kappa = setup[11]
lambda = setup[12]
n = 1000   # We have one thousand data points
m = 45   # We have m strategies
big_M = 1000
seed_numbers = 30

30

In [7]:
#Printing parameter settings
println("Parameter settings:")
println("form_data: ", form_data)
println("alpha: ", alpha)
println("alpha2:", alpha2)
println("beta: ", beta)
println("beta2: ", beta2)
println("gamma: ", gamma)
println("gamma2: ", gamma2)
println("rho: ", rho)
println("rho2: ", rho2)
println("sigma: ", sigma)
println("kappa: ", kappa)
println("lambda: ", lambda)
println("n: ", n)
println("m: ", m)
println("seed_numbers: ", seed_numbers)

Parameter settings:
form_data: newsvendor
alpha: 0.3
alpha2:0.8
beta: 1
beta2: 2
gamma: 1
gamma2: 3
rho: 0.75
rho2: 0.5
sigma: 0.1
kappa: 0.05
lambda: 0.1
n: 1000
m: 45
seed_numbers: 30


产生1000组实验数据，并绘制可视化结果

n=1000,m=35

In [ ]:
z_EWO

In [23]:
#Initializing results table
#Results = Array{Union{Missing, String, Int64, Float64}}(missing, 3*seed_numbers, 15+6)    #创建一个空数组。为啥有四种变量？？？？？？
println("Number 1")
seed_number = 1   ### 一个是seed_number, 一个是seed_numbers
#Running models for specified number of seeds to generate results
Sum = 0
for seed_number = 1:seed_numbers
	println()   #空行，显得更美观
	println("seed_number: ", seed_number)

	#Initializing parameter settings in results table
	#Results[3*(seed_number-1)+1,1:15] = [form_data,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,kappa,lambda,n,m,seed_number]   
	#Results[3*(seed_number-1)+2,1:15] = [form_data,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,kappa,lambda,n,m,seed_number]
	#Results[3*(seed_number-1)+3,1:15] = [form_data,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,kappa,lambda,n,m,seed_number]

	#Generating data
	x_data,z_data,y_data,y_min_data,y_max_data,d_data,d2_data = GenData(n,m,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,form_data,seed_number)
    
    #println(length(x_data))
    #println(length(z_data))
    #println(length(y_data))
    #println(length(d_data))
    #println(length(d2_data))
	#Solving Estimate-While-Optimize model
	solution_EWO = EWO(x_data,z_data,y_data,d_data,d2_data,kappa,lambda,big_M)

	#Saving solution and plotting parameters for Estimate-While-Optimize model
	a_EWO = solution_EWO[3]
    #println(a_EWO)
	b_EWO = solution_EWO[4]
    #println(b_EWO)
    #c_EWO = solution_EWO[5]
    b2_EWO = solution_EWO[5]
    #println(b2_EWO)
	z_EWO = solution_EWO[6]   # The value of z_EWO, which is a matrix
    #println(z_EWO)
    x_EWO = 0   # Define x_EWO
    Z_EWO = 0   # Define Z_EWO
    for i=1:length(z_EWO)   # Iterate all items in z_EWO and find the corresponding x_EWO and Z_EWO
        if z_EWO[i] ==1   # Note the index
            if rem(i,9)==0
                index_1 = 9
            else
                index_1 = i - fld(i,9)*9   # fld: floor the result
            end
            #index_1 = i - fld(i,9)*9   # fld: floor the result
            index_2 = cld(i,9)   # cld: ceil the result
            x_EWO = x_EWO + d_data[index_2]
            Z_EWO = Z_EWO + d2_data[index_1]   # Change needed!!
            break
        end
    end
	#x_EWO = dot(d_data,z_EWO)
    #Z_EWO = dot(d2_data,z_EWO)
	y_EWO = Result(y_min_data,y_max_data,x_EWO,Z_EWO,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,form_data)  
	#x_EWO_range = collect(x_EWO - kappa:(2*kappa/100):x_EWO + kappa)
    #z_EWO_range = collect(Z_EWO - kappa:(2*kappa/100):Z_EWO + kappa)
    #=
    println(length(a_EWO))
    println(length(x_EWO_range))
    println(length(b_EWO))
    println(length(b2_EWO))
    println(length(z_EWO_range))
    =#
	#y_EWO_range = a_EWO*ones(length(x_EWO_range)) + b_EWO*x_EWO_range + b2_EWO*z_EWO_range

	#Writing Estimate-While-Optimize results to results table
	#Results[3*(seed_number-1)+1,12:17] = ["EWO",a_EWO,b_EWO,0,x_EWO[1],y_EWO[1]]

	#Printing Estimate-While-Optimize results
	println("EWO")
	println("a: ", a_EWO)
	println("b: ", b_EWO)
    #println("c: ", c_EWO)
    println("b2: ",b2_EWO)
	println("x: ", x_EWO)
    println("z: ",Z_EWO)
	println("y: ", y_EWO)
    Sum = Sum + y_EWO

end
#Writing results table to text file
#writedlm("Results2.txt", Results)
### If wanna plot, delete for and end, make the seed_number a fixed number
	
#Plotting data   Why cannot plot here?Because our seed number is changing at first, so the software does not know how to draw it 
println("The average y value is: ",Sum/5)
#=
gr()
p = plot(x_data,y_data,seriestype=:scatter,title="Objective",xlabel="Decision",ylabel="Outcome",label="Data",legendfontsize=5,markersize=2,markercolor="grey")
p = plot!(x_EWO_range,y_EWO_range,label="EWO",linewidth=2,linecolor="red")      ###Why sometimes plot, sometimes plot!??????
plot(p)
=#

Number 1

seed_number: 1
Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10813 rows, 9815 columns and 182395 nonzeros
Model fingerprint: 0x0bbd24b5
Variable types: 2372 continuous, 7443 integer (7443 binary)
Coefficient statistics:
  Matrix range     [6e-04, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 2e+03]
Presolve removed 3302 rows and 3302 columns
Presolve time: 0.59s
Presolved: 7511 rows, 6513 columns, 132747 nonzeros
Variable types: 2372 continuous, 4141 integer (4115 binary)
Found heuristic solution: objective 0.8928257
Found heuristic solution: objective 300.3009738

Root relaxation: objective 1.000000e+03, 3127 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumb

Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10813 rows, 9815 columns and 182395 nonzeros
Model fingerprint: 0xc96609e1
Variable types: 2372 continuous, 7443 integer (7443 binary)
Coefficient statistics:
  Matrix range     [7e-05, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-04, 2e+03]
Presolve removed 3289 rows and 3289 columns
Presolve time: 0.70s
Presolved: 7524 rows, 6526 columns, 133660 nonzeros
Variable types: 2372 continuous, 4154 integer (4129 binary)
Found heuristic solution: objective 300.6940438

Root relaxation: objective 1.000000e+03, 3638 iterations, 0.10 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1000.00000    0  130  

Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10813 rows, 9815 columns and 182395 nonzeros
Model fingerprint: 0xeb593ed6
Variable types: 2372 continuous, 7443 integer (7443 binary)
Coefficient statistics:
  Matrix range     [5e-04, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-04, 2e+03]
Presolve removed 3289 rows and 3289 columns
Presolve time: 0.69s
Presolved: 7524 rows, 6526 columns, 133667 nonzeros
Variable types: 2372 continuous, 4154 integer (4133 binary)
Found heuristic solution: objective 20.8436538

Root relaxation: objective 1.000000e+03, 3429 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1000.00000    0   92   

seed_number: 11
Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10813 rows, 9815 columns and 182395 nonzeros
Model fingerprint: 0x64e19593
Variable types: 2372 continuous, 7443 integer (7443 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-04, 2e+03]
Presolve removed 3290 rows and 3290 columns
Presolve time: 0.76s
Presolved: 7523 rows, 6525 columns, 134646 nonzeros
Variable types: 2372 continuous, 4153 integer (4129 binary)
Found heuristic solution: objective 200.8027436

Root relaxation: objective 1.000000e+03, 3571 iterations, 0.13 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1000.0

seed_number: 14
Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10813 rows, 9815 columns and 182395 nonzeros
Model fingerprint: 0xd2b662cc
Variable types: 2372 continuous, 7443 integer (7443 binary)
Coefficient statistics:
  Matrix range     [2e-03, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-04, 2e+03]
Presolve removed 3310 rows and 3310 columns
Presolve time: 0.77s
Presolved: 7503 rows, 6505 columns, 133258 nonzeros
Variable types: 2372 continuous, 4133 integer (4109 binary)
Found heuristic solution: objective 110.5548814

Root relaxation: objective 1.000000e+03, 3148 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1000.0

Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10813 rows, 9815 columns and 182395 nonzeros
Model fingerprint: 0xb341d7de
Variable types: 2372 continuous, 7443 integer (7443 binary)
Coefficient statistics:
  Matrix range     [9e-04, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-04, 2e+03]
Presolve removed 3326 rows and 3326 columns
Presolve time: 0.76s
Presolved: 7487 rows, 6489 columns, 79122 nonzeros
Variable types: 2372 continuous, 4117 integer (4095 binary)
Found heuristic solution: objective 6.9036142
Found heuristic solution: objective 500.2067985

Root relaxation: objective 1.000000e+03, 2624 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/N

seed_number: 20
Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10813 rows, 9815 columns and 182395 nonzeros
Model fingerprint: 0x79e8861c
Variable types: 2372 continuous, 7443 integer (7443 binary)
Coefficient statistics:
  Matrix range     [4e-04, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-04, 2e+03]
Presolve removed 3290 rows and 3290 columns
Presolve time: 0.68s
Presolved: 7523 rows, 6525 columns, 132976 nonzeros
Variable types: 2372 continuous, 4153 integer (4131 binary)
Found heuristic solution: objective 0.9152916
Found heuristic solution: objective 300.2791737

Root relaxation: objective 1.000000e+03, 3497 iterations, 0.11 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Be

seed_number: 23
Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10813 rows, 9815 columns and 182395 nonzeros
Model fingerprint: 0xab84d871
Variable types: 2372 continuous, 7443 integer (7443 binary)
Coefficient statistics:
  Matrix range     [5e-04, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-05, 2e+03]
Presolve removed 3275 rows and 3275 columns
Presolve time: 0.68s
Presolved: 7538 rows, 6540 columns, 134209 nonzeros
Variable types: 2372 continuous, 4168 integer (4143 binary)
Found heuristic solution: objective 100.8179144
Found heuristic solution: objective 600.1317747

Root relaxation: objective 1.000000e+03, 3271 iterations, 0.06 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    

Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10813 rows, 9815 columns and 182395 nonzeros
Model fingerprint: 0x98650136
Variable types: 2372 continuous, 7443 integer (7443 binary)
Coefficient statistics:
  Matrix range     [3e-05, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-05, 2e+03]
Presolve removed 3257 rows and 3257 columns
Presolve time: 0.72s
Presolved: 7556 rows, 6558 columns, 136286 nonzeros
Variable types: 2372 continuous, 4186 integer (4155 binary)
Found heuristic solution: objective 10.7955004

Root relaxation: objective 1.000000e+03, 3828 iterations, 0.12 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1000.00000    0  127   

seed_number: 29
Academic license - for non-commercial use only - expires 2023-08-31
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10813 rows, 9815 columns and 182395 nonzeros
Model fingerprint: 0x3ad99909
Variable types: 2372 continuous, 7443 integer (7443 binary)
Coefficient statistics:
  Matrix range     [6e-04, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-03, 2e+03]
Presolve removed 3281 rows and 3281 columns
Presolve time: 0.74s
Presolved: 7532 rows, 6534 columns, 79493 nonzeros
Variable types: 2372 continuous, 4162 integer (4135 binary)
Found heuristic solution: objective 18.8792975

Root relaxation: objective 1.000000e+03, 2702 iterations, 0.10 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1000.000

In [24]:
println("The average y value is: ",Sum/seed_numbers)

The average y value is: 0.8532114484842487


# Benchmark Models

Linear Regression(2-variable situation)

In [9]:
seed_number = 5
x_data,z_data,y_data,y_min_data,y_max_data,d_data,d2_data = GenData(n,m,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,form_data,seed_number)
# Combine three columns of data together
com_data = hcat(x_data,z_data,y_data)
df = DataFrame(x_data = com_data[:,1],z_data = com_data[:,2],y_data = com_data[:,3])   # convert the data to DataFrame format
# Train test split
using Lathe.preprocess: TrainTestSplit
train, test = TrainTestSplit(df,.8)   # 80% of data points belong to the training set while 20% of them belong to the test set
println(test)
#Train the model
lin_mol = @formula(y_data ~  0 + x_data + z_data)
linearRegressor = lm(lin_mol, train)
# Prediction
#ypredicted_test = predict(linearRegressor, test)
#y_opti = maximum(ypredicted_test)   #The maximum value of linear regression

190×3 DataFrame
 Row │ x_data      z_data      y_data    
     │ Float64     Float64     Float64   
─────┼───────────────────────────────────
   1 │ 0.507445    0.226989    0.727035
   2 │ 0.549537    0.564746    0.529664
   3 │ 0.0672854   0.938366    0.300119
   4 │ 0.623887    0.998966    0.130094
   5 │ 0.981007    0.915211    0.0801982
   6 │ 0.726677    0.580604    0.399987
   7 │ 0.338763    0.713085    0.455856
   8 │ 0.456525    0.842494    0.330787
   9 │ 0.469683    0.346731    0.664355
  10 │ 0.907929    0.375568    0.51584
  11 │ 0.0287065   0.1149      0.914515
  12 │ 0.505638    0.828574    0.316731
  13 │ 0.203418    0.616344    0.554433
  14 │ 0.203009    0.690193    0.481502
  15 │ 0.0132257   0.692022    0.496023
  16 │ 0.436707    0.137501    0.807477
  17 │ 0.682332    0.2442      0.665647
  18 │ 0.535795    0.451215    0.596327
  19 │ 0.113291    0.450744    0.650759
  20 │ 0.817831    0.631639    0.363664
  21 │ 0.357511    0.585087    0.541705
  22 │ 0.568297   

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y_data ~ 0 + x_data + z_data

Coefficients:
───────────────────────────────────────────────────────────────────
           Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────
x_data  0.611707   0.0357208  17.12    <1e-55   0.54159    0.681823
z_data  0.191606   0.0353979   5.41    <1e-07   0.122123   0.261088
───────────────────────────────────────────────────────────────────

In [10]:
typeof(test)

DataFrame

In [11]:
# Delete rows that contain illegal elements
tbl = Tables.rowtable(test)
i = 0
del_list = []   # Store the rows that should be deleted
for row in Tables.rows(tbl)
    i = i+1
    if ((row[1]<0.1 || row[1]>0.9) || (row[2]<0.1 || row[2]>0.9))
        push!(del_list,i)
    end
end

delete!(test, del_list)
println(test)       
    

128×3 DataFrame
 Row │ x_data    z_data    y_data   
     │ Float64   Float64   Float64  
─────┼──────────────────────────────
   1 │ 0.507445  0.226989  0.727035
   2 │ 0.549537  0.564746  0.529664
   3 │ 0.726677  0.580604  0.399987
   4 │ 0.338763  0.713085  0.455856
   5 │ 0.456525  0.842494  0.330787
   6 │ 0.469683  0.346731  0.664355
   7 │ 0.505638  0.828574  0.316731
   8 │ 0.203418  0.616344  0.554433
   9 │ 0.203009  0.690193  0.481502
  10 │ 0.436707  0.137501  0.807477
  11 │ 0.682332  0.2442    0.665647
  12 │ 0.535795  0.451215  0.596327
  13 │ 0.113291  0.450744  0.650759
  14 │ 0.817831  0.631639  0.363664
  15 │ 0.357511  0.585087  0.541705
  16 │ 0.568297  0.304808  0.661847
  17 │ 0.683039  0.876981  0.244004
  18 │ 0.108222  0.666244  0.505809
  19 │ 0.714785  0.475181  0.480722
  20 │ 0.613488  0.62879   0.40749
  21 │ 0.412     0.846831  0.319642
  22 │ 0.176124  0.425067  0.690022
  23 │ 0.478723  0.106915  0.853429
  24 │ 0.688801  0.466161  0.505652
  25 │ 0.5

In [12]:
ypredicted_test = predict(linearRegressor, test)
ypredicted_test

128-element Vector{Union{Missing, Float64}}:
 0.35390005087899146
 0.4443638617828177
 0.5557601288429121
 0.34385456715877766
 0.4406863907418941
 0.353743894092184
 0.46806179072233206
 0.24252746132314557
 0.2564267131568733
 0.29348258894693385
 0.46417742566526027
 0.4142048154305434
 0.15566599637351985
 ⋮
 0.6451025201812997
 0.4503955266943567
 0.6207571184093524
 0.4382265069681402
 0.5587216109788773
 0.32996610058630027
 0.5620054695758891
 0.2873270837215406
 0.41445686868098147
 0.6395541049986466
 0.18945135191436765
 0.10214764330251234

In [17]:
x_data
x_data2 = []   # Store x_data that is greater than 0.1 and less than 0.9
z_data2 = []
for i=1:length(x_data)   # x_data and z_data share the same size
    if (x_data[i]>=0.1 && x_data[i]<=0.9)
        push!(x_data2,x_data[i])
    end
end

for i=1:length(z_data)   # x_data and z_data share the same size
    if (z_data[i]>=0.1 && z_data[i]<=0.9)
        push!(z_data2,z_data[i])
    end
end

max_xval = maximum(x_data2)
max_zval = maximum(z_data2)
y_max = 0.611707 *min_xval  +0.191606   *min_zval
println("The maximal value of linear regression is: ",y_max)

The maximal value of linear regression is: 0.08083424096792342


Quadratic Regression

In [19]:
seed_number = 5
x_data,z_data,y_data,y_min_data,y_max_data,d_data,d2_data = GenData(n,m,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,form_data,seed_number)
x2 = x_data.^2
z2 = z_data.^2
xz = x_data.*z_data
com_data = hcat(x_data,z_data,x2,xz,z2,y_data)
df = DataFrame(x_data = com_data[:,1],z_data = com_data[:,2],x2 = com_data[:,3],xz = com_data[:,4],z2 = com_data[:,5],y_data = com_data[:,6])   # convert the data to DataFrame format
# Train test split
using Lathe.preprocess: TrainTestSplit
train, test = TrainTestSplit(df,.8)   # 80% of data points belong to the training set while 20% of them belong to the test set
#Train the model
lin_mol = @formula(y_data ~  0 + x_data + z_data + x2 + xz + z2)
linearRegressor = lm(lin_mol, train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y_data ~ 0 + x_data + z_data + x2 + xz + z2

Coefficients:
────────────────────────────────────────────────────────────────────
           Coef.  Std. Error       t  Pr(>|t|)  Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────
x_data   1.94688   0.0761048   25.58    <1e-99    1.7975     2.09627
z_data   1.40582   0.0772688   18.19    <1e-61    1.25415    1.55749
x2      -1.30483   0.0863631  -15.11    <1e-44   -1.47436   -1.13531
xz      -1.37886   0.0681463  -20.23    <1e-73   -1.51262   -1.24509
z2      -1.2028    0.0873976  -13.76    <1e-38   -1.37436   -1.03125
────────────────────────────────────────────────────────────────────

The result above shows x^2,x*z, and z^2 are insignificant.

In [22]:
x_data2 = []   # Store x_data that is greater than 0.1 and less than 0.9
z_data2 = []
for i=1:length(x_data)   # x_data and z_data share the same size
    if (x_data[i]>=0.1 && x_data[i]<=0.9)
        push!(x_data2,x_data[i])
    end
end

for i=1:length(z_data)   # x_data and z_data share the same size
    if (z_data[i]>=0.1 && z_data[i]<=0.9)
        push!(z_data2,z_data[i])
    end
end

# Iterate all combinations of x_data2 and z_data2
d1 = length(x_data2)
d2 = length(z_data2)
y_valu = []
for i = 1:d1
    for j = 1:d2
        x2 = x_data2[i]^2
        xz = x_data2[i]*z_data2[j]
        z2 = z_data2[j]^2
        ymax = 1.94688*x_data2[i] + 1.40582*z_data2[j] -1.30483*x2 -1.37886*xz -1.2028*z2
        push!(y_valu,ymax)
    end
end
        
y_max = maximum(y_valu)
println("The maximal value of quadratic regression is: ",y_max)

The maximal value of quadratic regression is: 0.7686223490137885


Regression Tree

In [21]:
# importing the module
using DelimitedFiles

seed_number = 5
x_data,z_data,y_data,y_min_data,y_max_data,d_data,d2_data = GenData(n,m,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,form_data,seed_number)
writedlm("x_data2.txt", x_data)
writedlm("z_data2.txt", z_data)
writedlm("y_data2.txt", y_data)

In [26]:
y_data

1000-element Vector{Float64}:
 0.35814833918961225
 0.5966541067946219
 0.3927532236953652
 0.41889817107272165
 0.40221017748121396
 0.9003069454640562
 0.7600804433310858
 0.7033615224824911
 0.7597812965062587
 0.6573910679444235
 0.3252341851607607
 0.6712705296643187
 0.4767104242222609
 ⋮
 0.561808561971342
 0.9211655909556558
 0.30051752414962546
 0.39334770647883865
 0.3857563566717258
 0.6893817964635659
 0.8406005838299703
 0.7733780530194971
 0.5021295966735985
 0.7881396856706253
 0.4765212244315376
 0.31357413286526814

n=1000, m=45

In [15]:
#Initializing results table
#Results = Array{Union{Missing, String, Int64, Float64}}(missing, 3*seed_numbers, 15+6)    #创建一个空数组。为啥有四种变量？？？？？？
println("Number 1")
seed_number = 1   ### 一个是seed_number, 一个是seed_numbers
#Running models for specified number of seeds to generate results
Sum = 0
for seed_number = 1:seed_number
	println()   #空行，显得更美观
	println("seed_number: ", seed_number)

	#Initializing parameter settings in results table
	#Results[3*(seed_number-1)+1,1:15] = [form_data,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,kappa,lambda,n,m,seed_number]   
	#Results[3*(seed_number-1)+2,1:15] = [form_data,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,kappa,lambda,n,m,seed_number]
	#Results[3*(seed_number-1)+3,1:15] = [form_data,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,kappa,lambda,n,m,seed_number]

	#Generating data
	x_data,z_data,y_data,y_min_data,y_max_data,d_data,d2_data = GenData(n,m,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,form_data,seed_number)
    
    #println(length(x_data))
    #println(length(z_data))
    #println(length(y_data))
    #println(length(d_data))
    #println(length(d2_data))
	#Solving Estimate-While-Optimize model
	solution_EWO = EWO(x_data,z_data,y_data,d_data,d2_data,kappa,lambda,big_M)

	#Saving solution and plotting parameters for Estimate-While-Optimize model
	a_EWO = solution_EWO[3]
    #println(a_EWO)
	b_EWO = solution_EWO[4]
    #println(b_EWO)
    #c_EWO = solution_EWO[5]
    b2_EWO = solution_EWO[5]
    #println(b2_EWO)
	z_EWO = solution_EWO[6]   # The value of z_EWO, which is a matrix
    #println(z_EWO)
    x_EWO = 0   # Define x_EWO
    Z_EWO = 0   # Define Z_EWO
    for i=1:length(z_EWO)   # Iterate all items in z_EWO and find the corresponding x_EWO and Z_EWO
        if z_EWO[i] ==1   # Note the index
            if rem(i,9)==0
                index_1 = 9
            else
                index_1 = i - fld(i,9)*9   # fld: floor the result
            end
            #index_1 = i - fld(i,9)*9   # fld: floor the result
            index_2 = cld(i,9)   # cld: ceil the result
            x_EWO = x_EWO + d_data[index_2]
            Z_EWO = Z_EWO + d2_data[index_1]   # Change needed!!
            break
        end
    end
	#x_EWO = dot(d_data,z_EWO)
    #Z_EWO = dot(d2_data,z_EWO)
	y_EWO = Result(y_min_data,y_max_data,x_EWO,Z_EWO,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,form_data)  
	#x_EWO_range = collect(x_EWO - kappa:(2*kappa/100):x_EWO + kappa)
    #z_EWO_range = collect(Z_EWO - kappa:(2*kappa/100):Z_EWO + kappa)
    #=
    println(length(a_EWO))
    println(length(x_EWO_range))
    println(length(b_EWO))
    println(length(b2_EWO))
    println(length(z_EWO_range))
    =#
	#y_EWO_range = a_EWO*ones(length(x_EWO_range)) + b_EWO*x_EWO_range + b2_EWO*z_EWO_range

	#Writing Estimate-While-Optimize results to results table
	#Results[3*(seed_number-1)+1,12:17] = ["EWO",a_EWO,b_EWO,0,x_EWO[1],y_EWO[1]]

	#Printing Estimate-While-Optimize results
	println("EWO")
	println("a: ", a_EWO)
	println("b: ", b_EWO)
    #println("c: ", c_EWO)
    println("b2: ",b2_EWO)
	println("x: ", x_EWO)
    println("z: ",Z_EWO)
	println("y: ", y_EWO)
    Sum = Sum + y_EWO

end
#Writing results table to text file
#writedlm("Results2.txt", Results)
### If wanna plot, delete for and end, make the seed_number a fixed number
	
#Plotting data   Why cannot plot here?Because our seed number is changing at first, so the software does not know how to draw it 
println("The average y value is: ",Sum/seed_numbers)
#=
gr()
p = plot(x_data,y_data,seriestype=:scatter,title="Objective",xlabel="Decision",ylabel="Outcome",label="Data",legendfontsize=5,markersize=2,markercolor="grey")
p = plot!(x_EWO_range,y_EWO_range,label="EWO",linewidth=2,linecolor="red")      ###Why sometimes plot, sometimes plot!??????
plot(p)
=#

Number 1

seed_number: 1
Academic license - for non-commercial use only - expires 2022-08-28
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10813 rows, 9815 columns and 182395 nonzeros
Model fingerprint: 0x62976d60
Variable types: 2372 continuous, 7443 integer (7443 binary)
Coefficient statistics:
  Matrix range     [6e-04, 1e+03]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 2e+03]
Presolve removed 3302 rows and 3302 columns
Presolve time: 1.66s
Presolved: 7511 rows, 6513 columns, 132747 nonzeros
Variable types: 2372 continuous, 4141 integer (4115 binary)
Found heuristic solution: objective 0.8448762
Found heuristic solution: objective 300.2961176

Root relaxation: objective 1.000000e+03, 3286 iterations, 0.12 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumb

In [14]:
#=  Multi-line Comments
for i=1:5
    for j=1:5
        k = k+j
    end
end

println(p)
=#       

LoadError: UndefVarError: k not defined

n=4000, m=20

In [ ]:
#Initializing results table
#Results = Array{Union{Missing, String, Int64, Float64}}(missing, 3*seed_numbers, 15+6)    #创建一个空数组。为啥有四种变量？？？？？？
println("Number 1")
seed_number = 1   ### 一个是seed_number, 一个是seed_numbers
#Running models for specified number of seeds to generate results
Sum = 0
ite_num = 5
for seed_number = 1:ite_num
	println()   #空行，显得更美观
	println("seed_number: ", seed_number)

	#Initializing parameter settings in results table
	#Results[3*(seed_number-1)+1,1:15] = [form_data,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,kappa,lambda,n,m,seed_number]   
	#Results[3*(seed_number-1)+2,1:15] = [form_data,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,kappa,lambda,n,m,seed_number]
	#Results[3*(seed_number-1)+3,1:15] = [form_data,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,kappa,lambda,n,m,seed_number]

	#Generating data
	x_data,z_data,y_data,y_min_data,y_max_data,d_data,d2_data = GenData(n,m,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,sigma,form_data,seed_number)
    
    #println(length(x_data))
    #println(length(z_data))
    #println(length(y_data))
    #println(length(d_data))
    #println(length(d2_data))
	#Solving Estimate-While-Optimize model
	solution_EWO = EWO(x_data,z_data,y_data,d_data,d2_data,kappa,lambda,big_M)

	#Saving solution and plotting parameters for Estimate-While-Optimize model
	a_EWO = solution_EWO[3]
    #println(a_EWO)
	b_EWO = solution_EWO[4]
    #println(b_EWO)
    #c_EWO = solution_EWO[5]
    b2_EWO = solution_EWO[5]
    #println(b2_EWO)
	z_EWO = solution_EWO[6]   # The value of z_EWO, which is a matrix
    #println(z_EWO)
    x_EWO = 0   # Define x_EWO
    Z_EWO = 0   # Define Z_EWO
    for i=1:length(z_EWO)   # Iterate all items in z_EWO and find the corresponding x_EWO and Z_EWO
        if z_EWO[i] ==1   # Note the index
            if rem(i,9)==0
                index_1 = 9
            else
                index_1 = i - fld(i,9)*9   # fld: floor the result
            end
            #index_1 = i - fld(i,9)*9   # fld: floor the result
            index_2 = cld(i,9)   # cld: ceil the result
            x_EWO = x_EWO + d_data[index_2]
            Z_EWO = Z_EWO + d2_data[index_1]   # Change needed!!
            break
        end
    end
	#x_EWO = dot(d_data,z_EWO)
    #Z_EWO = dot(d2_data,z_EWO)
	y_EWO = Result(y_min_data,y_max_data,x_EWO,Z_EWO,alpha,alpha2,beta,beta2,gamma,gamma2,rho,rho2,form_data)  
	#x_EWO_range = collect(x_EWO - kappa:(2*kappa/100):x_EWO + kappa)
    #z_EWO_range = collect(Z_EWO - kappa:(2*kappa/100):Z_EWO + kappa)
    #=
    println(length(a_EWO))
    println(length(x_EWO_range))
    println(length(b_EWO))
    println(length(b2_EWO))
    println(length(z_EWO_range))
    =#
	#y_EWO_range = a_EWO*ones(length(x_EWO_range)) + b_EWO*x_EWO_range + b2_EWO*z_EWO_range

	#Writing Estimate-While-Optimize results to results table
	#Results[3*(seed_number-1)+1,12:17] = ["EWO",a_EWO,b_EWO,0,x_EWO[1],y_EWO[1]]

	#Printing Estimate-While-Optimize results
	println("EWO")
	println("a: ", a_EWO)
	println("b: ", b_EWO)
    #println("c: ", c_EWO)
    println("b2: ",b2_EWO)
	println("x: ", x_EWO)
    println("z: ",Z_EWO)
	println("y: ", y_EWO)
    Sum = Sum + y_EWO

end
#Writing results table to text file
#writedlm("Results2.txt", Results)
### If wanna plot, delete for and end, make the seed_number a fixed number
	
#Plotting data   Why cannot plot here?Because our seed number is changing at first, so the software does not know how to draw it 
println("The average y value is: ",Sum/seed_numbers)
#=
gr()
p = plot(x_data,y_data,seriestype=:scatter,title="Objective",xlabel="Decision",ylabel="Outcome",label="Data",legendfontsize=5,markersize=2,markercolor="grey")
p = plot!(x_EWO_range,y_EWO_range,label="EWO",linewidth=2,linecolor="red")      ###Why sometimes plot, sometimes plot!??????
plot(p)
=#

# Draft(Ignore it)

In [57]:
# Creating a 1D Array
Array1 = [1, 2, 3, 4, "Hello", "Geeks"]
 
# Adding element at the end
Array2 = ["Welcome"]
push!(Array1,"Welcome")
println(Array1)
append!(Array1, Array2)
println(Array1)
println(Array1,"Welcome")
d2 = collect(0.1:0.1:0.9)
println(d2)

Any[1, 2, 3, 4, "Hello", "Geeks", "Welcome"]
Any[1, 2, 3, 4, "Hello", "Geeks", "Welcome", "Welcome"]
Any[1, 2, 3, 4, "Hello", "Geeks", "Welcome", "Welcome"]Welcome
[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


In [58]:
s = [1 2;3 4]
println(length(s))   # The total # of elements
println(size(s))   # The size of s

#x_binding = @variable(model, [1:2, 1:3], base_name = "x")
#x_binding[1,1]

z = @variable(model, [1:2,1:2], Bin, base_name = 'z')
@constraint(model,sum(z[i,j] for i=1:2 for j=1:2) == 1)

4
(2, 2)


z[1,1] + z[2,1] + z[1,2] + z[2,2] == 1.0

In [13]:
B = [1 2; 3 4; 5 6; 7 8; 9 10]
for i=1:5
    for j=1:2
        if B[i,j] == 3
            println("We find the solution!")
        end
    end
end

We find the solution!


In [63]:
using Pkg #In-built package manager of Julia
Pkg.add("LinearAlgebra")    #Analogous to numpy package in Python
Pkg.add("Plots")    #Add Plots.jl framework for plotting
Pkg.add("JuMP")     #Add mathematical optimization package
Pkg.add("GLPK")     #Add solver
Pkg.add("IJulia")   #Backend for Jupyter interactive environment
Pkg.add("PyPlot")   #Julia interface to matplotlib.pyplot in Python

   Resolving package versions...
  No Changes to `C:\Users\czq13\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\czq13\.julia\environments\v1.6\Manifest.toml`
Precompiling project...
  ✓ Plots
  ✓ StatsPlots
  2 dependencies successfully precompiled in 62 seconds (231 already precompiled, 3 skipped during auto due to previous errors)
  1 dependency precompiled but a different version is currently loaded. Restart julia to access the new version
   Resolving package versions...
  No Changes to `C:\Users\czq13\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\czq13\.julia\environments\v1.6\Manifest.toml`
Precompiling project...
  ✓ Plots
  ✓ StatsPlots
  2 dependencies successfully precompiled in 59 seconds (231 already precompiled, 3 skipped during auto due to previous errors)
  1 dependency precompiled but a different version is currently loaded. Restart julia to access the new version
   Resolving package versions...
  No Changes to `C:\Users\czq13\.juli

In [10]:
using LinearAlgebra
using JuMP
using GLPK
using Plots

model = Model()
set_optimizer(model, GLPK.Optimizer)

#Define variables
@variable(model, x, lower_bound = 0, integer = true)
@variable(model, y, lower_bound = 0, integer = true)

#Define Constraints
@constraint(model, 3x+2y<=66)
@constraint(model, 9x+4y<=180)
@constraint(model, 2x+10y<=200)

#Define Objective
@objective(model, Max, 90x+75y)

#Run the opimization
optimize!(model)

In [11]:
obj_val = objective_value(model)
x_val = value.(x)
y_val = value.(y)
println(string("The objective value is:",obj_val))
println(string("The x value is: ",x_val))
println(string("The y value is: ",y_val))

The objective value is:2250.0
The x value is: 10.0
The y value is: 18.0


In [12]:
for i in index_x
  println("x[$i] = ", JuMP.value(x[i]))
end

2-element Vector{VariableRef}:
 noname
 noname

In [15]:
model = Model()
set_optimizer(model,GLPK.Optimizer)

# Define variables
@variable(model,x[1:2]>=0,Int)

# Define constraints
@constraint(model,3x[1]+2x[2]<=66)   # Pro 1: Don't need to print "*" anymore
@constraint(model,9x[1]+4x[2]<=180)
@constraint(model,2x[1]+10x[2]<=200)

# Define Objective
@objective(model,Max,90x[1]+75x[2])

# Run the optimization
optimize!(model)

In [19]:
for i=1:2
  println("x[$i] = ", value.(x[i]))   # Pro2: Can use the format to print the result
end

x[1] = 10.0
x[2] = 18.0


In [20]:
x = [1 2 3]
for i=1:length(x)
    println("x[$i] = ",x[i])
end

x[1] = 1
x[2] = 2
x[3] = 3
